In [7]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
from PIL import Image
import io
import os
import glob
from datetime import datetime
from google.cloud import vision
from google.cloud.vision import types

In [8]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bin/ROKS-78e082836307.json"

plt.figure(figsize=(50, 40))
plt.style.use('dark_background')

images = glob.glob('data/info/original/*.jpg')

<Figure size 3600x2880 with 0 Axes>

In [9]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in texts[0].bounding_poly.vertices])

    return texts[0].description

In [10]:
def change_to_number(s_power):
    list_of_numbers = re.findall(r'\d+', s_power)
    return int(''.join(list_of_numbers))

In [11]:
def change_to_list(chars):
    tmp_texts = list()
    s = ""
    for c in chars:
        if c != '\n':
            s += c
        else:
            tmp_texts.append(s)
            s = ""
    
    return tmp_texts

In [15]:
start_time = datetime.now()

# crop & convert
for image in images:
    initial = Image.open(image)
    width = initial.size[0]
    height = initial.size[1]
    
    path, file_name = os.path.split(image)

    #name
    croped = initial.crop((width*0.295, height*0.15, width*0.8, height*0.2))
    croped.save('data/info/croped/name/' + file_name)

    #kill
    croped_1 = initial.crop((width*0.579, height*0.28, width*0.64, height*0.44))
    croped_2 = initial.crop((width*0.655, height*0.28, width*0.72, height*0.44))
    
    croped_1.save('data/info/croped/kill/0_' + file_name)
    croped_2.save('data/info/croped/kill/1_' + file_name)


    for j in range(2):
        img_kill = cv2.imread('data/info/croped/kill/'+ str(j) + '_' + file_name)
        gray = cv2.cvtColor(img_kill, cv2.COLOR_BGR2GRAY)
        structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

        imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
        imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

        imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
        gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

        img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

        ret, img_thresh = cv2.threshold(
            img_blurred, 
            190,
            255,
            cv2.THRESH_BINARY
        )
        cv2.imwrite('data/info/convert/kill/'+ str(j) + '_' + file_name, img_thresh)

    #dead & support
    croped = initial.crop((width*0.20, height*0.23, width*0.80, height*0.98))
    croped.save('data/info/croped/ds/' + file_name)

end_time = datetime.now()

print (end_time - start_time)

0:00:24.722333


In [17]:
start_time = datetime.now()
# detect
name = list()
power = list()
all_kill = list()
T5_kill = list()
T4_kill = list()
T3_kill = list()
T2_kill = list()
T1_kill = list()
dead = list()
support = list()

image_path = list()

n = 1 

p = re.compile('\d{4}/\d{2}/\d{2}')

tmp = list()

re.sub('original', 'croped/name', images[0])
for image in images:
    
    path, file_name = os.path.split(image)
    image_path.append(image)
    
    #name
    chars = detect_text('data/info/croped/name/' + file_name)
    tmp_name = change_to_list(chars)
    
    name.append(tmp_name[0])
    power.append(change_to_number(tmp_name[1]))
    all_kill.append(change_to_number(tmp_name[2]))
    
    #kill    
    tmp_kill = list()
    for j in range(2):
        chars = detect_text('data/info/convert/kill/'+ str(j) + '_' + file_name)
        tmp_kill.append(change_to_list(chars))

    T1_kill.append(change_to_number(tmp_kill[0][0]))
    T2_kill.append(change_to_number(tmp_kill[1][0]))
    T3_kill.append(change_to_number(tmp_kill[0][1]))
    T4_kill.append(change_to_number(tmp_kill[1][1]))
    T5_kill.append(change_to_number(tmp_kill[0][2]))
     
    #dead & support
    chars = detect_text('data/info/croped/ds/' + file_name)
    tmp_ds = change_to_list(chars)
    
    dead.append(change_to_number(tmp_ds[tmp_ds.index('전사')+1]))
    support.append(change_to_number(tmp_ds[tmp_ds.index('원조')+1]))
    
    print(n)
    n += 1
        
df1 = pd.DataFrame({'name': name,
                   'power':power,
                   'all_kill':all_kill,
                   'T1_kill':T1_kill,
                   'T2_kill':T2_kill,
                   'T3_kill':T3_kill,
                   'T4_kill':T4_kill,
                   'T5_kill':T5_kill,
                   'dead':dead,
                   'support':support})

df1.to_excel('./data/info/result5.xlsx')

end_time = datetime.now()

print (end_time - start_time)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
0:13:54.804077


In [171]:
print(image_path[-1])

./data/info/original/Screenshot_20200727-203102_Rise of Kingdoms.jpg
